In [26]:
user_name = 'celia'

In [27]:
import sys
sys.path.append('/Users/{}/GitHub/mouse_bandit/data_preprocessing_code'.format(user_name))
sys.path.append('/Users/{}/GitHub/mouse_bandit'.format(user_name))
import support_functions as sf
import numpy as np
import pandas as pd
import scipy as sp
import bandit_preprocessing as bp
from sklearn import preprocessing
import os
import behavior_database
import shutil
import ipywidgets as widgets
from IPython.display import display

In [28]:
record_df = pd.read_csv('/Users/{}/GitHub/mouse_bandit/session_record.csv'.format(user_name),index_col=0)
celia_session_record = True

### First time we ran this, had to add 'shay' to the old data base and add a new column for Markov
    if need to troubleshoot or add trials from older data, use creating_behavior_database notebook

In [29]:
print(record_df.shape)
record_df.tail(3)

(807, 18)


,Owner,Session ID,Mouse ID,Date,Phase,Markov,Left Reward Prob,Right Reward Prob,Block Range Min,Block Range Max,No. Trials,No. Blocks,No. Rewards,p(high Port),Decision Window Duration,Min Inter-trial-interval,Left Solenoid Duration,Right Solenoid Duration
804,shay,03102017_K9,K9,2017-03-10 00:00:00,2.0,0,0.9,0.1,50.0,50.0,329.0,5.0,249.0,0.79,2.0,1.0,35.0,35.0
805,shay,03102017_Q43,Q43,2017-03-10 00:00:00,2.0,0,0.9,0.1,50.0,50.0,556.0,9.0,476.0,0.90,2.0,1.0,35.0,35.0
806,shay,03102017_Q45,Q45,2017-03-10 00:00:00,2.0,0,0.1,0.9,50.0,50.0,642.0,10.0,509.0,0.88,2.0,1.0,35.0,35.0


## Add all days (multi mice) to celia_session_record

In [30]:
if celia_session_record == True:
    record_df = record_df.drop(record_df.index[:]) # if want to reset and start with (for new session record)
    
    MOUSE = ['Baby','Mustard','Raspberry','Sky','Wine', 'Dill']
    ROOT = '/Users/{}/Dropbox (HMS)/Celia_BehaviorFiles/'.format(user_name)
    root_mouse={}
    for mouse in MOUSE: # make list of all mouse directories
        temp_dict = {mouse:os.path.join(ROOT,mouse)}
        root_mouse.update(temp_dict)

    date_folders = os.listdir(root_mouse[MOUSE[0]]) # assuming all mice have same dates 
    date_folders = date_folders[1:]
    count=0
    for mouse in MOUSE:
        for date in date_folders:
            curr_dir = os.path.join(root_mouse[mouse], date)
            record_df = behavior_database.add_session(curr_dir, record_df, 'celia')
            count+=1
    record_df.to_csv(path_or_buf='/Users/{}/GitHub/mouse_bandit/celia_session_record.csv'.format(user_name))

In [31]:
count - 60 # estimate for number of sessions

942

In [32]:
record_df.shape # true number of sessions in record

(927, 18)

## Moving files around
    Use MovingFilesAround notebook for moving data outside of new structure

In [33]:
# files from Celia

root_dir = '/Users/{}/Dropbox (HMS)/Celia_BehaviorFiles/'.format(user_name)
MOUSE = ['Baby','Mustard','Raspberry','Sky','Wine']
root_mouse={}
for mouse in MOUSE: # make list of all mouse directories
    temp_dict = {mouse:os.path.join(root_dir,mouse)}
    root_mouse.update(temp_dict)
date_folders = os.listdir(root_mouse[MOUSE[0]]) # assuming all mice have same dates 
date_folders = date_folders[1:]

dist_dir = '/Users/{}/GitHub/mouse_bandit/data/trial_data'.format(user_name)

for mouse in MOUSE:
    for date in date_folders:
        for file in os.listdir(os.path.join(root_mouse[mouse], date)):

            if not file[0] == '.':
                file_name = os.path.join(root_mouse[mouse],date,file)
                if 'trials.csv' in file:
                    shutil.copy(file_name, dist_dir)
                    #print(file_name)
                else:
                    pass
                    #print(date_folders)
                    #print(root_mouse[mouse])
                    #raise ValueError('In a hidden folder')
                    
MOUSE = ['Dill']
root_mouse={}
for mouse in MOUSE: # make list of all mouse directories
    temp_dict = {mouse:os.path.join(root_dir,mouse)}
    root_mouse.update(temp_dict)
date_folders = os.listdir(root_mouse[MOUSE[0]]) # assuming all mice have same dates 
date_folders = date_folders[1:]

dist_dir = '/Users/{}/GitHub/mouse_bandit/data/trial_data'.format(user_name)

for mouse in MOUSE:
    for date in date_folders:
        for file in os.listdir(os.path.join(root_mouse[mouse], date)):

            if not file[0] == '.':
                file_name = os.path.join(root_mouse[mouse],date,file)
                if 'trials.csv' in file:
                    shutil.copy(file_name, dist_dir)
                    #print(file_name)
                else:
                    pass
                    #print(date_folders)
                    #print(root_mouse[mouse])
                    #raise ValueError('In a hidden folder')
        

## Make feature matrix for each condition type

In [34]:
db = pd.read_csv('/Users/{}/GitHub/mouse_bandit/celia_session_record.csv'.format(user_name),index_col=0)
db.shape # just to double check size updated

(927, 18)

#### all markov sessions with >0.7 performance
Need to fix this to collect same proportion from each condition

In [35]:
r = db[(db['Markov']==1)]
r = r[r['p(high Port)'] > 0.7].copy()
r = r[r['No. Blocks']>0.0].copy()
session_names = r['Session ID'].values

In [36]:
'''
load in trial data
'''
columns = ['Elapsed Time (s)','Since last trial (s)','Trial Duration (s)','Port Poked',
           'Right Reward Prob','Left Reward Prob','Reward Given',
          'center_frame','decision_frame', 'Block ID']

root_dir = '/Users/{}/GitHub/mouse_bandit/data/trial_data'.format(user_name)
save_dir = '/Users/{}/Dropbox (HMS)/mouse_bandit/'.format(user_name)

trial_df = []

for session in session_names:
    full_name = session + '_trials.csv'
    
    path_name = os.path.join(root_dir,full_name)
    
    trial_df.append(pd.read_csv(path_name,names=columns))

mouse_ids = r['Mouse ID'].values

In [37]:
progress = widgets.IntProgress(min=0, max=len(trial_df), step=1)
display(progress)
for i,df in enumerate(trial_df):
    curr_feature_matrix = bp.create_feature_matrix(df,10,mouse_ids[i],session_names[i],feature_names='Default')
    
    if i == 0:
        master_matrix = curr_feature_matrix.copy()
    else:
        master_matrix = master_matrix.append(curr_feature_matrix)
    progress.value+=1

In [38]:
master_matrix.to_csv(os.path.join(save_dir,'markov_master.csv'),index=True)

#### Pull sessions from each reward condition

In [39]:
r = db[(db['Markov']==0)]
r = r[((r['Left Reward Prob'] == 0.7) |  (db['Left Reward Prob'] == 0.3))].copy()
r = r[r['No. Blocks']>0.0].copy()
session_names = r['Session ID'].values

/Users/celia/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  


In [40]:
# set threshold for inclusion based on reward parameters
threshold = r['p(high Port)'].mean() - 2 * r['p(high Port)'].std()

In [41]:
threshold

0.44563518271368135

In [42]:
r = r[r['p(high Port)'] > threshold].copy()

In [43]:
'''
load in trial data
'''
columns = ['Elapsed Time (s)','Since last trial (s)','Trial Duration (s)','Port Poked',
           'Right Reward Prob','Left Reward Prob','Reward Given',
          'center_frame','decision_frame', 'Block ID']

root_dir = '/Users/{}/GitHub/mouse_bandit/data/trial_data'.format(user_name)
save_dir = '/Users/{}/Dropbox (HMS)/mouse_bandit/'.format(user_name)

trial_df = []

for session in session_names:
    full_name = session + '_trials.csv'
    
    path_name = os.path.join(root_dir,full_name)
    
    trial_df.append(pd.read_csv(path_name,names=columns))

mouse_ids = r['Mouse ID'].values

In [44]:
progress = widgets.IntProgress(min=0, max=len(trial_df), step=1)
display(progress)
for i,df in enumerate(trial_df):
    curr_feature_matrix = bp.create_feature_matrix(df,10,mouse_ids[i],session_names[i],feature_names='Default')
    
    if i == 0:
        master_matrix = curr_feature_matrix.copy()
    else:
        master_matrix = master_matrix.append(curr_feature_matrix)
    progress.value+=1

IndexError: index 75 is out of bounds for axis 0 with size 75

In [ ]:
master_matrix.to_csv(os.path.join(save_dir,'blocks_7030_cb.csv'),index=True)